In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
from source.datasets.fast_datasets import *
from source.utils.load_utils import split_and_load
#import torch

genre_dataset = GTZANFastDataset()

deezer_dataset = DeezerFastDataset()

train_dataloader, val_dataloader = split_and_load(genre_dataset, workers=8, batch_size=4, split_size=0.75)

In [50]:
from source.models_task_specific.mb_classification_refactor import MusicBertClassifier

sound_features = genre_dataset.get_features_size(log=True)
num_classes = 10
evals = 1

print("Using %d features"%sound_features)

model = MusicBertClassifier(num_classes, RNN=False, num_encoder_layers=12).cuda()
#model.BERT.load_pretrained()
model.load_model("models/music_bert_pretrain_classification.pth")

== Item shapes ==
song_features:  (2999, 64)
encoded_class:  (1,)
Using 64 features


In [51]:
curve_bert = model.train_model(train_dataloader, val_dataloader, epochs = 100, eval_per_epoch=evals)

Eval Loss (18699 steps) 0.7298            


In [52]:
%matplotlib widget
import numpy as np

from source.utils.plot_utils import plot_curve

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth[box_pts:-box_pts]

loss = smooth(model.loss_curve.cpu().numpy(), 20)

plot_curve(loss, 1, color="red")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
%matplotlib widget

plot_curve(saved_curve, 1, color="red")
plot_curve(saved_curve_2, 1, color="blue")
plot_curve(saved_curve_3, 1, color="green")
plot_curve(saved_curve_4, 1, color="yellow")
plot_curve(model.validation_curve.cpu().numpy(), 1, color="purple")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …